# CICF 2022




# Week 2

## Cloud site

HTCondor Central Manager Virtual Machine

1. Use Chameleon's Python API to check for an existing or create a new lease.
1. Use Chameleon's Python API to create and start a VM with `node_type="compute_cascadelake_r"`.
1. Associate a floating IP address with the VM.
1. Install HTCondor v8.8 and Pegasus on the VM.
1. Configure HTCondor to run under `CentralManager`, `Submit`, and `Execute` roles.

# Links

* Main – https://www.chameleoncloud.org/
* Python API – https://python-chi.readthedocs.io/en/latest/
* Webinars – https://www.chameleoncloud.org/talks/

# Note:

1. Set the Project name to `CHI-220973`
1. Set the site `CHI@TACC`
1. Use image `CC-Ubuntu18.04`

In [32]:
# Python Imports
import chi
from chi import lease

In [33]:
# Configure CHI Project
chi.set('project-name','CHI-220973')
chi.use_site('CHI@TACC')
from chi import lease

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


In [3]:
# Code for Week 2

# Create a lease with reservations for 1 node and 1 floating IP.
user = "khushich"
clease = f"{user}-cmtest"
node_type = "compute_cascadelake_r"
creservations = []

chi.lease.add_node_reservation(
    creservations,node_type= node_type,count = 1)
chi.lease.add_fip_reservation(creservations, count =1)
start_date, end_date = chi.lease.lease_duration(days=1)
chi.lease.create_lease(
    clease, creservations, start_date=start_date, end_date=end_date)
#floatingip = chi.network.get_free_floating_ip(True)


{'created_at': '2022-07-08 21:43:58',
 'updated_at': '2022-07-08 21:44:00',
 'id': '5ceb80f1-3fb0-4acb-b2a4-cf5a743a9dc3',
 'name': 'khushich-cmtest',
 'user_id': '7639ed6a1005085b9b08e58e6e3bf171b245dc1e67acf9dcb12b41c932c0c050',
 'project_id': '9898a31fb33841d29ed99ba84a54e10b',
 'start_date': '2022-07-08T21:45:00.000000',
 'end_date': '2022-07-09T21:44:00.000000',
 'trust_id': 'e24b6a8da0e1481cb4b7fb4dea560c71',
 'status': 'PENDING',
 'degraded': False,
 'reservations': [{'created_at': '2022-07-08 21:43:58',
   'updated_at': '2022-07-08 21:44:00',
   'id': '1807434e-acc3-42cc-8ddf-a492550c3953',
   'lease_id': '5ceb80f1-3fb0-4acb-b2a4-cf5a743a9dc3',
   'resource_id': '5c163c19-a677-41af-aab6-01d909c8a77a',
   'resource_type': 'physical:host',
   'status': 'pending',
   'missing_resources': False,
   'resources_changed': False,
   'hypervisor_properties': '',
   'resource_properties': '["==", "$node_type", "compute_cascadelake_r"]',
   'before_end': 'default',
   'on_start': 'default

In [6]:
# Start a virtual machine 
# First, launch a bare metal instance
from chi.lease import get_node_reservation
from chi.server import create_server

server_name = "khushi-server"
reservation_id = get_node_reservation(clease)
server = create_server(server_name, reservation_id=reservation_id, image_name = "CC-Ubuntu18.04")

In [7]:
from chi.server import list_servers

list_servers()

[<Server: khushi-server>, <Server: nnersisy_server>]

# Attach floating IP to VM and wait for SSH

In [8]:
from chi.server import associate_floating_ip, wait_for_tcp
server_id = server.id
ip = associate_floating_ip(server_id,floating_ip_address = None)

#Wait for SSH connectivity
wait_for_tcp(ip, port=22)

In [9]:
# Install and configure the software
# Steps to install and configure should be in a file called config/vm-config.sh
# Copy vm-config.sh to the VM and execute it with the floating IP as a command line argument
from chi import ssh
with ssh.Remote(ip) as conn:
    print("Installing and Configuring the Software")
    conn.put("config/vm-config/vm-config.sh")
    conn.run("bash vm-config.sh " + ip)

Installing and Configuring the Software


/opt/conda/lib/python3.9/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 129.114.109.253: b'9c75bc26c9bdbebe184b457592037e06'
  warnings.warn(


OK
Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:1 https://research.cs.wisc.edu/htcondor/repo/ubuntu/8.8 bionic InRelease [3981 B]
Get:5 https://research.cs.wisc.edu/htcondor/repo/ubuntu/8.8 bionic/main amd64 Packages [16.7 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2313 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2653 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [806 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1212 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [19.0 kB]
Get:12 http://nova.clouds.archive.ubunt

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 11.0 MB in 2s (6531 kB/s)
Selecting previously unselected package x11-common.
(Reading database ... 63272 files and directories currently installed.)
Preparing to unpack .../00-x11-common_1%3a7.7+19ubuntu7.1_all.deb ...
dpkg-query: no packages found matching nux-tools
Unpacking x11-common (1:7.7+19ubuntu7.1) ...
Selecting previously unselected package libxss1:amd64.
Preparing to unpack .../01-libxss1_1%3a1.2.2-1_amd64.deb ...
Unpacking libxss1:amd64 (1:1.2.2-1) ...
Preparing to unpack .../02-python2.7_2.7.17-1~18.04ubuntu1.7_amd64.deb ...
Unpacking python2.7 (2.7.17-1~18.04ubuntu1.7) over (2.7.17-1~18.04ubuntu1.6) ...
Preparing to unpack .../03-libpython2.7-stdlib_2.7.17-1~18.04ubuntu1.7_amd64.deb ...
Unpacking libpython2.7-stdlib:amd64 (2.7.17-1~18.04ubuntu1.7) over (2.7.17-1~18.04ubuntu1.6) ...
Preparing to unpack .../04-python2.7-minimal_2.7.17-1~18.04ubuntu1.7_amd64.deb ...
Unpacking python2.7-minimal (2.7.17-1~18.04ubuntu1.7) over (2.7.17-1~18.04ubuntu1.6) ...
Preparing to

Created symlink /etc/systemd/system/multi-user.target.wants/condor.service → /lib/systemd/system/condor.service.
--2022-07-08 22:47:12--  http://download.pegasus.isi.edu/pegasus/gpg.txt
Resolving download.pegasus.isi.edu (download.pegasus.isi.edu)... Warning: apt-key output should not be parsed (stdout is not a terminal)
128.9.37.80
Connecting to download.pegasus.isi.edu (download.pegasus.isi.edu)|128.9.37.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1680 (1.6K) [text/plain]
Saving to: ‘STDOUT’

     0K .                                                     100%  127M=0s

2022-07-08 22:47:12 (127 MB/s) - written to stdout [1680/1680]



OK
deb [arch=amd64] http://download.pegasus.isi.edu/pegasus/ubuntu bionic main
Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 http://download.pegasus.isi.edu/pegasus/ubuntu bionic InRelease
Hit:1 https://research.cs.wisc.edu/htcondor/repo/ubuntu/8.8 bionic InRelease
Get:5 http://download.pegasus.isi.edu/pegasus/ubuntu bionic Release [1829 B]
Get:6 http://download.pegasus.isi.edu/pegasus/ubuntu bionic Release.gpg [833 B]
Hit:7 http://nova.clouds.archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://nova.clouds.archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:9 http://download.pegasus.isi.edu/pegasus/ubuntu bionic/main amd64 Packages [877 B]
Fetched 3539 B in 1s (2796 B/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  ca-certificates-java default-jre default-jre-he

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 119 MB in 4s (27.4 MB/s)
Selecting previously unselected package fontconfig.
(Reading database ... 63763 files and directories currently installed.)
Preparing to unpack .../00-fontconfig_2.12.6-0ubuntu2_amd64.deb ...
Unpacking fontconfig (2.12.6-0ubuntu2) ...
Selecting previously unselected package libice6:amd64.
Preparing to unpack .../01-libice6_2%3a1.0.9-2_amd64.deb ...
Unpacking libice6:amd64 (2:1.0.9-2) ...
Selecting previously unselected package libsm6:amd64.
Preparing to unpack .../02-libsm6_2%3a1.2.2-1_amd64.deb ...
Unpacking libsm6:amd64 (2:1.2.2-1) ...
Selecting previously unselected package libxxf86vm1:amd64.
Preparing to unpack .../03-libxxf86vm1_1%3a1.1.4-1_amd64.deb ...
Unpacking libxxf86vm1:amd64 (1:1.1.4-1) ...
Selecting previously unselected package java-common.
Preparing to unpack .../04-java-common_0.68ubuntu1~18.04.1_all.deb ...
Unpacking java-common (0.68ubuntu1~18.04.1) ...
Selecting previously unselected package libcups2:amd64.
Preparing to unpack .../05-

In [10]:
# Checks to ensure software is configured correctly
# To check, do pegasus-version, condor_version, condor_q, and condor_status
with ssh.Remote(ip) as conn:
    print("Pegasus Version is:")
    conn.run("pegasus-version")

    print("Condor Version is:")
    conn.run("condor_version")

    print("Running condor_q:")
    conn.run("condor_q")

    print("Condor Status is:")
    conn.run("condor_status")

Pegasus Version is:
5.0.2
Condor Version is:
$CondorVersion: 8.8.17 Mar 11 2022 BuildID: Debian-8.8.17-1+b1 PackageID: 8.8.17-1+b1 Debian-8.8.17-1+b1 $
$CondorPlatform: X86_64-Ubuntu_18.04 $
Running condor_q:


-- Schedd: khushi-server.novalocal : <129.114.109.253:9618?... @ 07/08/22 22:49:19
OWNER BATCH_NAME      SUBMITTED   DONE   RUN    IDLE   HOLD  TOTAL JOB_IDS

Total for query: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended 
Total for cc: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended 
Total for all users: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended

Condor Status is:
Name                           OpSys      Arch   State     Activity LoadAv Mem   ActvtyTime

slot1@khushi-server.novalocal  LINUX      X86_64 Unclaimed Idle      0.000 1999  0+00:00:03
slot2@khushi-server.novalocal  LINUX      X86_64 Unclaimed Idle      0.000 1999  0+00:00:44
slot3@khushi-server.novalocal  LINUX      X86_64 Unclaimed Idle    

In [ ]:
# Cleanup resources
chi.use_site('CHI@TACC')
chi.server.delete_server(server_id)
chi.lease.delete_lease(clease)

# Week 3

## Edge Site

1. Use Chameleon's Python API to check for an existing or create a new lease.
1. Use Chameleon's Python API to create and start a container (rajivmayani/condor8-arm64-plankifier-worker).
1. Set CONDOR_HOST to the VM's floating IP address from Week 2.

# Links

* Main – https://www.chameleoncloud.org/
* Python API – https://python-chi.readthedocs.io/en/latest/
* Webinars – https://www.chameleoncloud.org/talks/

In [11]:
# Code for Week 3
import chi
chi.set('project_name', 'CHI-220973')
chi.use_site('CHI@Edge')

from chi.lease import get_device_reservation
from chi.container import create_container

Now using CHI@Edge:
URL: https://chi.edge.chameleoncloud.org
Location: Distributed
Support contact: https://groups.google.com/g/chameleon-edge-users


In [18]:
# Create a lease with reservations for 1 raspberry pi
user = "khushich"
elease = f"{user}-edge"
machine_name="raspberrypi4-64"
container_name = "khushi-container"
lreservations = []

chi.lease.add_device_reservation(lreservations,count=1, machine_name=machine_name)

start_date, end_date = chi.lease.lease_duration(days=1)
chi.lease.create_lease(elease,lreservations,start_date = start_date, end_date = end_date)
reservation_id = get_device_reservation(elease,machine_name=machine_name,count = None)

In [ ]:
# Create the Container
container = create_container(container_name, image = "rajivmayani/condor8-arm64-plankifier-worker",reservation_id=reservation_id)

In [24]:
# Configure the software already installed in the container
# Steps to configure should be in a file called config/edge-config/edge-config.sh
# Copy edge-config.sh to the container and execute it with the floating IP of the central manager as a command line argument
print("Installing and Configuring the Software")
chi.container.upload(container_ref = "khushi-container", source = "config/edge-config/", dest = "/")
chi.container.execute(container_ref = "khushi-container", command = "bash /edge-config.sh " + ip)

Installing and Configuring the Software


{'output': 'E: Could not get lock /var/lib/dpkg/lock-frontend - open (11: Resource temporarily unavailable)\nE: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), is another process using it?\n/edge-config.sh: line 5: sudo: command not found\n/edge-config.sh: line 8: STARTD_ATTRS: command not found\nSent "Reconfig" command to local master\n',
 'exit_code': 0,
 'exec_id': None,
 'proxy_url': None}

In [25]:
# Checks to ensure software is configured correctly
print("Checking to ensure software is configured correctly:")
chi.container.execute(container_ref = "khushi-container", command = "condor_status")

Checking to ensure software is configured correctly:


{'output': '', 'exit_code': None, 'exec_id': None, 'proxy_url': None}

In [ ]:
# Cleanup resources
# chi.container.destroy_container(container_name)

# Week 4

1. Create a prototype Pegasus Workflow for the application.
1. Develop, run, and test it locally on your laptop.

# Links

* Doc – https://pegasus.isi.edu/documentation/
* Tutorial – https://pegasus.isi.edu/documentation/user-guide/tutorial.html
* YouTube – https://www.youtube.com/channel/UCwJQln1CqBvTJqiNr9X9F1Q 

In [26]:
# Change site back to the cloud site
import chi
chi.set('project_name', 'CHI-220973')
chi.use_site('CHI@TACC')

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


In [ ]:
# Code for Week 4
# Code should be stored in a directory called plankifier-wf

# Week 5

1. Migrate the workflow to the resources created on Chameleon.
1. Run the workflow on Chameleon.

# Links

* Doc – https://pegasus.isi.edu/documentation/
* Tutorial – https://pegasus.isi.edu/documentation/user-guide/tutorial.html
* YouTube – https://www.youtube.com/channel/UCwJQln1CqBvTJqiNr9X9F1Q 

In [31]:
# Code for Week 5
# Copy plankifier-wf directory to the Chameleon VM
# Start the Pegasus workflow in the VM
# Print the results of the workflow
with ssh.Remote(ip) as conn:
    print(conn)
    !tar zcf plankifier-wf.tar.gz plankifier-wf
    conn.put("plankifier-wf.tar.gz")
    conn.run("rm -rf plankifier-wf")
    conn.run("tar xvf plankifier-wf.tar.gz")
    conn.run("rm plankifier-wf.tar.gz")
    conn.run("cd plankifier-wf")
    !rm plankifier-wf.tar.gz
    conn.run("cd plankifier-wf ; rm -rf submit scratch output")
    conn.run("cd plankifier-wf ; ./pegasus-wf.py")
    
    print("Output:")
    conn.run("cat plankifier-wf/wf-output/counts.txt")  

<Connection host=129.114.109.253 user=cc>
plankifier-wf/
plankifier-wf/pegasus-wf.py
plankifier-wf/.ipynb_checkpoints/
plankifier-wf/.ipynb_checkpoints/workflow-checkpoint.sh
plankifier-wf/.ipynb_checkpoints/pegasus-wf-checkpoint.py
plankifier-wf/workflow.sh
plankifier-wf/bin/
plankifier-wf/bin/count.sh
plankifier-wf/bin/.ipynb_checkpoints/
plankifier-wf/bin/.ipynb_checkpoints/count-checkpoint.sh


2022.07.09 01:42:46.483 UTC: [WARNING]  Replica Catalog is either down or connection to it was never opened
2022.07.09 01:42:46.793 UTC:
2022.07.09 01:42:46.799 UTC:   -----------------------------------------------------------------------
2022.07.09 01:42:46.805 UTC:   File for submitting this DAG to HTCondor           : main-sh-0.dag.condor.sub
2022.07.09 01:42:46.810 UTC:   Log of DAGMan debugging messages                 : main-sh-0.dag.dagman.out
2022.07.09 01:42:46.815 UTC:   Log of HTCondor library output                     : main-sh-0.dag.lib.out
2022.07.09 01:42:46.821 UTC:   Log of HTCondor library error messages             : main-sh-0.dag.lib.err
2022.07.09 01:42:46.826 UTC:   Log of the life of condor_dagman itself          : main-sh-0.dag.dagman.log
2022.07.09 01:42:46.832 UTC:
2022.07.09 01:42:46.837 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with:
2022.07.09 01:42:46.848 UTC:   -------------------------------------------------------------

[#########################] 100.0% ..Success (Unready: 0, Completed: 12, Queued: 0, Running: 0, Failed: 0), Queued: 0, Running: 1, Failed: 0)eady: 8, Completed: 3, Queued: 0, Running: 1, Failed: 0)--]  25.0% ..Running (Unready: 8, Completed: 3, Queued: 0, Running: 1, Failed: 0)1;32m########-----------------]  33.3% ..Running (Unready: 6, Completed: 4, Queued: 2, Running: 0, Failed: 0)1, Failed: 0)mQueued: 2, Running: 0, Failed: 0)0m, Completed: 12, Queued: 0, Running: 0, Failed: 0)
Output:
      2 aphanizomenon
      6 asplanchna
      1 asterionella
      7 bosmina
      5 conochilus
      2 copepod_skins
      6 cyclops
     10 daphnia
      3 daphnia_skins
      5 diaphanosoma
      2 dinobryon
      2 dirt
      3 eudiaptomus
      3 filament
      8 fish
      4 fragilaria
      1 kellikottia
      4 leptodora
      1 maybe_cyano
      3 nauplius
      2 paradileptus
      1 synchaeta
     17 Unclassified
      1 unknown
      4 unknown_plankton
      2 uroglena


# Week 6

1. Prepare a poster for the application.


In [ ]:
# Cleanup resources

